<a href="https://colab.research.google.com/github/runmark/Andrew-Ng-Deep-Learning-notes/blob/master/supabase_storage_duckdb_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Supabase Storage and DuckDB: ⚡️ + 🦆 = 💚

## Table of Contents

0. [Setup and Installation](#Setup-and-Installation)
1. [Query Postgres Data and Export to Supabase Storage](#Query-Postgres-Data-and-Export-to-Supabase-Storage)
2. [Query Supabase Storage Directly with DuckDB](#Query-Supabase-Storage-Directly-with-DuckDB)
3. [Visualizing Results](#Visualizing-Results)

## Setup and Installation

### Install Packages
comment out the below block if you need to install the packages into your environment (if running locally or in Google Colab)

In [ ]:
# !pip install -r https://raw.githubusercontent.com/TylerHillery/supabase-storage-duckdb-demo/main/requirements.txt?token=GHSAT0AAAAAACPZEHJ7KIXE6CPID5MDMUE6ZQSAINQ

Then we will import the necessary packages

In [ ]:
import os
from datetime import datetime

import altair as alt
import duckdb
from dotenv import load_dotenv

This lines take environment variables from a .env file. If you are not using a .env file feel free to skip it.

In [ ]:
load_dotenv()

We will define some variables used for configuration management.

In [ ]:
# Postgres credentials
POSTGRES_HOST = os.getenv("POSTGRES_HOST")
POSTGRES_PORT = os.getenv("POSTGRES_PORT")
POSTGRES_DATABASE = os.getenv("POSTGRES_DATABASE")
POSTGRES_USERNAME = os.getenv("POSTGRES_USERNAME")
POSTGRES_PASSWORD = os.getenv("POSTGRES_PASSWORD")

# Supabase storage credentials
AWS_ACCESS_KEY_ID = os.getenv("AWS_ACCESS_KEY_ID")
AWS_SECRET_ACCESS_KEY = os.getenv("AWS_SECRET_ACCESS_KEY")
AWS_REGION = os.getenv("AWS_REGION")
BUCKET_URL = os.getenv("BUCKET_URL")
ENDPOINT_URL = os.getenv("ENDPOINT_URL")

DuckDB has native support for the [S3 API](https://duckdb.org/docs/extensions/httpfs/s3api.html) and we can use the DuckDB [Secrets Manager](https://duckdb.org/docs/configuration/secrets_manager) to store our credentials

In [ ]:
duckdb.sql(f"""
DROP SECRET IF EXISTS supabase_storage;
CREATE SECRET supabase_storage (
    TYPE S3,
    KEY_ID '{AWS_ACCESS_KEY_ID}',
    SECRET '{AWS_SECRET_ACCESS_KEY}',
    ENDPOINT '{ENDPOINT_URL}',
    REGION '{AWS_REGION}',
    URL_STYLE 'path'
)
""")

Load helpful DuckDB Postgres extension.

In [ ]:
duckdb.sql("INSTALL postgres")

duckdb.sql(f"""
ATTACH
    'dbname={POSTGRES_DATABASE}
    user={POSTGRES_USERNAME}
    host={POSTGRES_HOST}
    password={POSTGRES_PASSWORD}
    port={POSTGRES_PORT}'
AS postgres_db (TYPE POSTGRES, READ_ONLY)
""")

## Query Postgres Data and Export to Supabase Storage

Lets first query data in our Postgres database and look at some results

In [ ]:
select_pg_customers = """
SELECT
    id                              AS user_id,
    first_name                      AS user_first_name,
    last_name                       AS user_last_name,
    first_name || ' ' || last_name  AS user_full_name,
    CURRENT_DATE                    as loaded_at_date,
    CURRENT_TIMESTAMP               AS loaded_at_ts_utc
FROM
    postgres_db.customers
"""
duckdb.sql(select_pg_customers).df().head()

Now lets export those query results to our Supabase Storage bucket as a parquet file.

In [ ]:
duckdb.sql(f"""
COPY ({select_pg_customers})
TO '{BUCKET_URL}/customers/{datetime.now().strftime("%Y-%m-%d")}.parquet'
""")

You can do CSV file as well

In [ ]:
duckdb.sql(f"""
COPY ({select_pg_customers})
TO '{BUCKET_URL}/customers/{datetime.now().strftime("%Y-%m-%d")}.csv'
"""
)

You can also do partitioned copies

In [ ]:
duckdb.sql(f"""
COPY ({select_pg_customers})
TO 's3://postgres/customers' (
    FORMAT PARQUET,
    PARTITION_BY (loaded_at_date),
    OVERWRITE_OR_IGNORE true
)
""")

Lets do the same for the rest of the tables

In [ ]:
select_pg_orders = """
SELECT
    id                  AS order_id,
    user_id             AS user_id,
    order_date          AS order_date,
    status              AS order_status,
    CURRENT_DATE        AS loaded_at_date,
    CURRENT_TIMESTAMP   AS loaded_at_ts_utc
FROM
    postgres_db.orders
"""
duckdb.sql(select_pg_orders).df().head()

In [ ]:
duckdb.sql(f"""
COPY ({select_pg_orders})
TO '{BUCKET_URL}/orders/{datetime.now().strftime("%Y-%m-%d")}.parquet'
""")

In [ ]:
select_pg_payments = """
SELECT
    id                  AS payment_id,
    order_id            AS order_id,
    payment_method      AS payment_method,
    amount              AS order_amount_usd,
    CURRENT_DATE        AS loaded_at_date,
    CURRENT_TIMESTAMP   AS loaded_at_ts_utc
FROM
    postgres_db.payments
"""
duckdb.sql(select_pg_payments).df().head()

In [ ]:
duckdb.sql(f"""
COPY ({select_pg_payments})
TO '{BUCKET_URL}/payments/{datetime.now().strftime("%Y-%m-%d")}.parquet'
""")

## Query Supabase Storage Directly with DuckDB

Now the tables are loaded lets query the files from Supabase Storage. We can even do file globbing patterns and return the filename.

In [ ]:
select_from_bucket = f"""
SELECT
    filename,
    count(*) as record_count
FROM
    read_parquet('{BUCKET_URL}/orders/*.parquet', filename = true)
GROUP BY
    ALL
"""
duckdb.sql(select_from_bucket).show()

In [ ]:
join_tables = f"""
SELECT
    orders.order_date,
    orders.order_id,
    customers.user_full_name,
    orders.order_status,
    payments.payment_method,
    payments.order_amount_usd,
    customers.user_id,
    payments.payment_id
FROM
    read_parquet('{BUCKET_URL}/orders/*.parquet') AS orders
    LEFT JOIN read_parquet('{BUCKET_URL}/customers/*.parquet') AS customers
        ON orders.user_id = customers.user_id
    LEFT JOIN read_parquet('{BUCKET_URL}/payments/*.parquet') AS payments
        ON orders.order_id = payments.order_id
"""
orders_df = duckdb.sql(join_tables).df()

orders_df.head()

## Visualizing Results

DuckDB can return results in variety of formats such as:
- `.fetchall()` returns a list of tuples
- `.df()` returns a Pandas DataFrame
- `.pl()` returns a Polars DataFrame
- `.arrow()` return an Arrow Table
- `.fetchnumpy()` returns NumPy Arrays

This works nicely with several data viz libraries e.g. Matplotlib, Seaborn, Bokeh. I will be using Altair which is based on Vega-Lite

In [ ]:
monthly_sales = """
SELECT
    date_trunc('month', order_date) as order_month,
    sum(order_amount_usd) as sales
FROM
    orders_df
WHERE
    order_status != 'returned'
GROUP BY
    ALL
ORDER BY
    order_month
"""

chart = (
    alt.Chart(duckdb.sql(monthly_sales).df())
    .mark_line()
    .encode(
        x="order_month",
        y=alt.Y("sales", axis=alt.Axis(format="$,.2f")),
        tooltip=["order_month", "sales"],
    )
    .properties(width=750, height=400)
    .interactive()
)
chart

In [ ]:
monthly_sales_by_order_status = """
SELECT
    date_trunc('month', order_date)::string as order_month,
    order_status,
    sum(order_amount_usd) as sales
FROM
    orders_df
GROUP BY
    ALL
ORDER BY
   order_month
"""

chart = (
    alt.Chart(duckdb.sql(monthly_sales_by_order_status).df())
    .mark_bar()
    .encode(
        x="order_month",
        y=alt.Y("sales", axis=alt.Axis(format="$,.2f")),
        tooltip=["order_month", "sales", "order_status"],
        color="order_status",
    )
    .properties(width=750, height=400)
    .interactive()
)
chart